import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=480
cls_num=7
btch=50
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=FISH(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (175, 464)
first_test_shape= (175, 464)
classes_quantity= 7
tr_lbls=	 {1, 2, 3, 4, 5, 6, 7}
Count_labels= [26 25 28 21 22 25 28]
max(train_feature_Altitude)= 7.0
min(train_feature_Altitude)= -1.9506
first_train_sample=
 [ 7.0000e+00  1.9509e+00  1.9267e+00  1.8957e+00  1.8580e+00  1.8206e+00
  1.7903e+00  1.7676e+00  1.7313e+00  1.6952e+00  1.6589e+00  1.6230e+00
  1.5869e+00  1.5572e+00  1.5217e+00  1.4861e+00  1.4505e+00  1.4140e+00
  1.3777e+00  1.3415e+00  1.3054e+00  1.2699e+00  1.2348e+00  1.1989e+00
  1.1633e+00  1.1267e+00  1.0922e+00  1.0566e+00  1.0210e+00  9.8561e-01
  9.4955e-01  9.1484e-01  8.7873e-01  8.4270e-01  8.0675e-01  7.7089e-01
  7.3512e-01  6.9944e-01  6.6385e-01  6.2656e-01  5.9114e-01  5.5395e-01
  5.1871e-01  4.8163e-01  4.4643e-01  4.0963e-01  3.7273e-01  3.3589e-01
  3.0122e-01  2.6453e-01  2.2790e-01  1.9134e-01  1.5484e-01  1.1848e-01
  8.2080e-02  4.5819e-02  9.6418e-03 -2.6445e-02 -6.2437e-02 -9.8329e-02
 -1.3412e-01 -1.6979e-01 -2.053

In [3]:
np.shape(ecg)

(175, 481)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 7., 7., 7., 7., 7., 7.,
       7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.,
       7., 7., 7., 7., 7.])

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 26	2 25	3 28	4 21	5 22	6 25	7 28	
 max = 28


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (26, 481)
Up to class  2 train shape =  (51, 481)
Up to class  3 train shape =  (79, 481)
Up to class  4 train shape =  (100, 481)
Up to class  5 train shape =  (122, 481)
Up to class  6 train shape =  (147, 481)
Up to class  7 train shape =  (175, 481)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0 26 25 28 21 22 25 28]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 7., 7., 7., 7., 7., 7.,
       7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7.,
       7., 7., 7., 7., 7.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  2.0470912776915458
min magnitude class 1  =  -1.7067841463837912
after normalizing >>
max magnitude class 1  =  0.961349554710035
min magnitude class 1  =  -0.8796770785564421

 cls 2  >> 
max magnitude class 2  =  1.9995000000000018
min magnitude class 2  =  -1.737570959253758
after normalizing >>
max magnitude class 2  =  0.9380091931254746
min magnitude class 2  =  -0.8947759659844823

 cls 3  >> 
max magnitude class 3  =  2.1528521925548616
min magnitude class 3  =  -1.777414662371874
after normalizing >>
max magnitude class 3  =  1.0132182607872897
min magnitude class 3  =  -0.9143166566898705

 cls 4  >> 
max magnitude class 4  =  11.25984586666761
min magnitude class 4  =  -1.9420530969143601
after normalizing >>
max magnitude class 4  =  5.479593941637377
min magnitude class 4  =  -0.9950608766253729

 cls 5  >> 
max magnitude class 5  =  2.1211000000000007
min magnitude class 5  =  -1.7538854427257269
after normalizing >>
max magnitude cl

In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.9950608766253729
max tst = 5.479593941637377


In [14]:
len(wndws_test)

175

In [15]:
np.shape(wndws_test)

(175, 481)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7],
      dtype=int16)


<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(26, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (175, 480)
y_train =>  (175,)
X_test  =>  (175, 480)
y_test  =>  (175,)
X_valid  =>  (175, 480)
y_valid  =>  (175,)


In [22]:
y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [23]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [24]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
3/3 [==============================] - 4s 346ms/step - loss: 5.1381 - accuracy: 0.2000 - val_loss: 2.1831 - val_accuracy: 0.1429
Epoch 2/1000
3/3 [==============================] - 0s 28ms/step - loss: 2.0789 - accuracy: 0.1086 - val_loss: 2.1434 - val_accuracy: 0.1429
Epoch 3/1000
3/3 [==============================] - 0s 33ms/step - loss: 2.4527 - accuracy: 0.1143 - val_loss: 2.1637 - val_accuracy: 0.1600
Epoch 4/1000
3/3 [==============================] - 0s 28ms/step - loss: 1.9958 - accuracy: 0.2400 - val_loss: 2.1446 - val_accuracy: 0.1600
Epoch 5/1000
3/3 [==============================] - 0s 29ms/step - loss: 1.9876 - accuracy: 0.2057 - val_loss: 2.0862 - val_accuracy: 0.1600
Epoch 6/1000
3/3 [==============================] - 0s 28ms/step - loss: 2.1092 - accuracy: 0.2343 - val_loss: 2.1389 - val_accuracy: 0.1200
Epoch 7/1000
3/3 [==============================] - 0s 28ms/step - loss: 1.8269 - accuracy: 0.2743 - val_loss: 2.0755 - val_accuracy: 0.1257
Epoch 8/1000

3/3 [==============================] - 0s 29ms/step - loss: 2.8816e-05 - accuracy: 1.0000 - val_loss: 1.9024 - val_accuracy: 0.2571
Epoch 59/1000
3/3 [==============================] - 0s 30ms/step - loss: 2.5497e-05 - accuracy: 1.0000 - val_loss: 1.9025 - val_accuracy: 0.2571
Epoch 60/1000
3/3 [==============================] - 0s 30ms/step - loss: 2.2313e-05 - accuracy: 1.0000 - val_loss: 1.9017 - val_accuracy: 0.2571
Epoch 61/1000
3/3 [==============================] - 0s 30ms/step - loss: 1.7606e-05 - accuracy: 1.0000 - val_loss: 1.9157 - val_accuracy: 0.2571
Epoch 62/1000
3/3 [==============================] - 0s 30ms/step - loss: 1.7599e-05 - accuracy: 1.0000 - val_loss: 1.9048 - val_accuracy: 0.2571
Epoch 63/1000
3/3 [==============================] - 0s 31ms/step - loss: 1.2744e-05 - accuracy: 1.0000 - val_loss: 1.8990 - val_accuracy: 0.2571
Epoch 64/1000
3/3 [==============================] - 0s 29ms/step - loss: 1.2868e-05 - accuracy: 1.0000 - val_loss: 1.8909 - val_accuracy:

In [25]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [26]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

6/6 [==============================] - 0s 3ms/step
array([[ 9,  0,  0,  0,  1, 14,  0],
       [25,  0,  0,  0,  0,  0,  0],
       [ 7,  0, 10,  0,  0,  5,  0],
       [11,  0,  0,  0,  0, 18,  0],
       [ 2,  0,  0,  0, 20,  6,  0],
       [ 3,  0,  0,  0,  0, 22,  0],
       [12,  0,  0,  0,  0, 10,  0]], dtype=int64)
              precision    recall  f1-score   support

           2       0.13      0.38      0.19        24
           3       0.00      0.00      0.00        25
           4       1.00      0.45      0.62        22
           5       0.00      0.00      0.00        29
           6       0.95      0.71      0.82        28
           7       0.29      0.88      0.44        25
           8       0.00      0.00      0.00        22

    accuracy                           0.35       175
   macro avg       0.34      0.35      0.30       175
weighted avg       0.34      0.35      0.30       175



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
3/3 [==============================] - 2s 323ms/step - loss: 37.3251 - accuracy: 0.1200 - val_loss: 2.1748 - val_accuracy: 0.1600
Epoch 2/1000
3/3 [==============================] - 0s 39ms/step - loss: 2.1846 - accuracy: 0.2000 - val_loss: 2.1530 - val_accuracy: 0.1429
Epoch 3/1000
3/3 [==============================] - 0s 37ms/step - loss: 2.0407 - accuracy: 0.1257 - val_loss: 2.1444 - val_accuracy: 0.1486
Epoch 4/1000
3/3 [==============================] - 0s 37ms/step - loss: 1.9673 - accuracy: 0.2514 - val_loss: 2.1317 - val_accuracy: 0.1600
Epoch 5/1000
3/3 [==============================] - 0s 38ms/step - loss: 1.9411 - accuracy: 0.1943 - val_loss: 2.1254 - val_accuracy: 0.1429
Epoch 6/1000
3/3 [==============================] - 0s 38ms/step - loss: 1.7772 - accuracy: 0.3257 - val_loss: 2.0691 - val_accuracy: 0.1600
Epoch 7/1000
3/3 [==============================] - 0s 46ms/step - loss: 2.3912 - accuracy: 0.1600 - val_loss: 2.1412 - val_accuracy: 0.1771
Epoch 8/100

Epoch 59/1000
3/3 [==============================] - 0s 36ms/step - loss: 0.3384 - accuracy: 0.9086 - val_loss: 1.9090 - val_accuracy: 0.2400
Epoch 60/1000
3/3 [==============================] - 0s 36ms/step - loss: 0.0275 - accuracy: 0.9943 - val_loss: 1.8987 - val_accuracy: 0.2686
Epoch 61/1000
3/3 [==============================] - 0s 36ms/step - loss: 0.0605 - accuracy: 0.9829 - val_loss: 1.9048 - val_accuracy: 0.2800
Epoch 62/1000
3/3 [==============================] - 0s 37ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 1.8843 - val_accuracy: 0.2686
Epoch 63/1000
3/3 [==============================] - 0s 37ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 1.8517 - val_accuracy: 0.2857
Epoch 64/1000
3/3 [==============================] - 0s 37ms/step - loss: 6.6615e-04 - accuracy: 1.0000 - val_loss: 1.8317 - val_accuracy: 0.3143
Epoch 65/1000
3/3 [==============================] - 0s 37ms/step - loss: 2.8072e-04 - accuracy: 1.0000 - val_loss: 1.8414 - val_accuracy: 0.222

In [28]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

6/6 [==============================] - 0s 4ms/step
array([[11,  0,  5,  8,  0,  0,  0],
       [ 3,  0,  1, 21,  0,  0,  0],
       [ 1,  0, 21,  0,  0,  0,  0],
       [ 4,  0,  2, 23,  0,  0,  0],
       [10,  0, 12,  6,  0,  0,  0],
       [22,  0,  2,  1,  0,  0,  0],
       [ 3,  0,  1,  1,  0,  0, 17]], dtype=int64)
              precision    recall  f1-score   support

           2       0.20      0.46      0.28        24
           3       0.00      0.00      0.00        25
           4       0.48      0.95      0.64        22
           5       0.38      0.79      0.52        29
           6       0.00      0.00      0.00        28
           7       0.00      0.00      0.00        25
           8       1.00      0.77      0.87        22

    accuracy                           0.41       175
   macro avg       0.29      0.43      0.33       175
weighted avg       0.28      0.41      0.31       175



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
